In [1]:
import pybullet as p
import numpy as np
import sys
import os
import json
import open3d as o3d
import matplotlib.pyplot as plt
import time
from PIL import Image

sys.path.append("../")
from env.ycb_scene import SimulatedYCBEnv
from utils.grasp_checker import ValidGraspChecker
from utils.utils import *
from utils.planner import GraspPlanner

pybullet build time: May 20 2022 19:44:17


In [2]:
from utils.planner import GraspPlanner
planner = GraspPlanner()

def expert_plan(goal_pose, world=False, visual=False):
    if world:
        pos, orn = env._get_ef_pose()
        ef_pose_list = [*pos, *orn]
    else:
        ef_pose_list = [0, 0, 0, 0, 0, 0, 1]
    goal_pos = [*goal_pose[:3], *ros_quat(goal_pose[3:])]

    solver = planner.plan(ef_pose_list, goal_pos, path_length=30)
    if visual:
        path_visulization(solver)
    path = solver.getSolutionPath().getStates()
    planer_path = []
    for i in range(len(path)):
        waypoint = path[i]
        rot = waypoint.rotation()
        action = [waypoint.getX(), waypoint.getY(), waypoint.getZ(), rot.w, rot.x, rot.y, rot.z]
        planer_path.append(action)

    return planer_path

def path_visulization(ss):
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    x = []
    y = []
    z = []
    for i in range(len(ss.getSolutionPath().getStates())):
        state = ss.getSolutionPath().getStates()[i]
        x.append(state.getX())
        y.append(state.getY())
        z.append(state.getZ())
    ax.plot(x, y, z, color='gray', label='Curve')

    ax.scatter(x, y, z, c=z, cmap='jet', label='Points')
    plt.show()
    
def execute_motion_plan(env, plan, execute=False, gripper_set="close", repeat=100):
    """
    Executes a series of movements in a robot environment based on the provided plan.

    Parameters:
    - env: The current robot environment, providing inverse kinematics solving and stepping capabilities.
    - plan: A plan containing target positions, each position is a list with coordinates and orientation.
    - execute: A boolean indicating whether to execute the actions. If False, only resets the robot's joint positions without stepping through the environment.
    """
    for i in range(len(plan)):
        # Set target position using world frame based coordinates
        next_pos = plan[i]
        jointPoses = env._panda.solveInverseKinematics(next_pos[:3], ros_quat(next_pos[3:]))
        if gripper_set == "close":
            jointPoses[6] = 0.85
        else:
            jointPoses[6] = 0.0
        jointPoses = jointPoses[:7].copy()  # Consider only the first 7 joint positions
                
        if execute:
            # Execute the action and obtain the observation
            obs = env.step(jointPoses, config=True, repeat=repeat)[0]
            # print("JointPoses = ", jointPoses)
        else:
            # Only reset the robot's joint positions
            env._panda.reset(joints=jointPoses)
            # time.sleep(0.5)
            # print("JointPoses = ", jointPoses)
        

def check_pose_difference(current_pose, target_pose, tolerance=0.01):
    """
    檢查物體的當前姿態與目標姿態之間的差異是否在許可範圍內。
    
    參數:
        current_pose (numpy.ndarray): 物體的當前4x4轉換矩陣。
        target_pose (numpy.ndarray): 物體的目標4x4轉換矩陣。
        tolerance (float): 許可的最大差異範圍。
        
    返回:
        bool: 當前姿態是否達到目標姿態的範圍內。
    """
    # 計算從當前姿態到目標姿態的變換矩陣
    diff_matrix = np.dot(np.linalg.inv(current_pose), target_pose)
    
    # 從差異矩陣中提取平移差異
    translation_diff = np.linalg.norm(diff_matrix[:3, 3])
    print("translation_diff = ", translation_diff)
    
    # 從差異矩陣中提取旋轉差異，並確保輸入值位於[-1, 1]範圍內
    cos_angle = (np.trace(diff_matrix[:3, :3]) - 1) / 2
    cos_angle_clamped = np.clip(cos_angle, -1, 1)  # 限制cos_angle的值在[-1, 1]範圍內
    # get the rotation difference in radian
    rotation_diff = np.arccos(cos_angle_clamped)
    print("rotation_diff = ", rotation_diff)
    
    # 檢查平移和旋轉是否都在許可的範圍內
    return translation_diff <= tolerance and rotation_diff <= tolerance



In [3]:
### 將open3d下的點雲轉到world座標
world_frame_pose = np.array([[ 1.,    0.,    0.,   -0.05],
                            [ 0.,    1.,    0.,    0.  ],
                            [ 0.,    0.,    1.,   -0.65],
                            [ 0.,    0.,    0.,    1.  ]])

init_ef_mat = np.array([[-1.98785608e-01,  7.23231525e-01,  6.61377686e-01,  1.06898375e-01],
                        [9.80042993e-01,  1.46612626e-01,  1.34240345e-01, -9.29623842e-02],
                        [1.20530092e-04,  6.74863616e-01, -7.37942468e-01, -0.3],
                        [0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

left_view_ef_mat = np.array([[ 0.98757027,  0.02243495,  0.15556875,  0.45691898],
                            [ 0.14573556, -0.501431,   -0.85283533,  0.36891946],
                            [ 0.05887368,  0.86490672, -0.49846791, -0.3],
                            [ 0.,          0.,          0.,          1.]])

right_view_ef_mat = np.array([[ 0.98691477, -0.16087768,  0.010845,    0.46446365],
                            [-0.10023915, -0.55945926,  0.82277424, -0.28816143],
                            [-0.12629867, -0.81309514, -0.56826485, -0.3],
                            [ 0.,          0.,          0.,          1.]])

cam_offset = np.eye(4)
# 先轉到pybullet座標後再往上移動0.13變相機座標
cam_offset[:3, 3] = (np.array([0., 0.1186, -0.0191344123493]))
# open3d 和 pybullet中的coordinate要對z旋轉180度才會一樣
cam_offset[:3, :3] = np.array([[-1, 0, 0],
                                [0, -1, 0],
                                [0, 0, 1]])

intrinsic_matrix = np.array([[320, 0, 320],
                             [0, 320, 320],
                             [0, 0, 1]])

### 轉換關係
origin_camera2world = cam_offset@ np.linalg.inv(init_ef_mat)@ world_frame_pose
left_camera2world = cam_offset@ np.linalg.inv(left_view_ef_mat)@ world_frame_pose
right_camera2world = cam_offset@ np.linalg.inv(right_view_ef_mat)@ world_frame_pose

In [4]:
'''
get data file name in json file and load mesh in pybullet
then reset robot and object position
'''
# tcp server
# target_place_name = "Mustard_1.0"
target_place_name = "Milk_1.0"
# target_place_name = "BBQSauce_1.0"
# target_place_name = "hope_dataset"
# target_place_name = start_server_target_name('127.0.0.1', 11111)
file = os.path.join("../object_index", 'contact_plane_object.json')   # 此json檔可以自己改
with open(file) as f: file_dir = json.load(f)
file_dir = file_dir[target_place_name]     #只取json檔中的"test"
file_dir = [f[:-5] for f in file_dir]
test_file_dir = list(set(file_dir))

env = SimulatedYCBEnv()
env._load_index_objs(test_file_dir)      #597

#ycb scene 中的 init 可以定義target_fixed, true代表target object不會自由落下 (第一次呼叫cabinet設定即可)
state = env.cabinet(save=False, enforce_face_target=True)  

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce GTX 1080 Ti/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 525.147.05
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 525.147.05
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce GTX 1080 Ti/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
numActiveThreads = 0
stopping threads
destroy semaphore
semaphore destroyed
Thread with taskId 0 exiting
Thread TERMINATED
destroy main semaphore
main semaphore destroye

In [5]:
'''
single release: (只在num_object = 1有用) true為以自己設定的角度放在桌上; (多object也可用)false就是pack放在桌上
if_stack: true代表旁邊有東西會擋住掉落下來的物體
'''
num_object = 1
single_release = False
if_stack = True
vis = True

state = env.cabinet(save=False, reset_free=True, num_object=num_object, if_stack=if_stack, single_release=single_release)

single_release： False
>>>> target name: Milk_1.0


In [6]:
placed_obj = {}
placed_idx = np.where(np.array(env.placed_objects))[0]
placed_name = np.array(env.obj_indexes)[np.where(np.array(env.placed_objects))]
for i in range(num_object):
    placed_obj[placed_idx[i]] = placed_name[i]
print(placed_obj)

{0: 'Milk_1.0'}


# 將final中轉換後的所有的grasp pose load進來

In [11]:
parent_directory = os.path.dirname(os.getcwd()) 
print(parent_directory)

/home/user/contact_graspnet_ws/src/contact-graspnet-ros


In [13]:
new_pred_grasps_cam_place = np.load(os.path.join(parent_directory, 'results/new_pred_grasps_cam_place.npy'), allow_pickle=True)
print(new_pred_grasps_cam_place.shape)
scores = np.load(os.path.join(parent_directory, 'results/scores.npy'), allow_pickle=True)
print(scores.shape)

(58, 4, 4)
(58,)


### 擺放target object到place pose

In [14]:
# relative pose of z axis rotation
env._panda.reset() 

checker_list = []
execute =  False
for i in range(len(new_pred_grasps_cam_place)):
    grasp_pose = new_pred_grasps_cam_place[i]
    final_place_pose_z_bias = grasp_pose
    END_POINT = final_place_pose_z_bias
    END_POINT = END_POINT
    env.draw_ef_coordinate(END_POINT, 3)
    plan = expert_plan(pack_pose(END_POINT), world=True, visual=False)
    execute_motion_plan(env, plan, execute=execute, gripper_set = "open", repeat=200)
    # check the pose is correct or not
    checker = check_pose_difference(env._get_ef_pose(mat=True), END_POINT, tolerance=0.01)
    checker_list.append(checker)
    print(f"checker: {checker}")
    if(env._panda.check_for_collisions() == True):
        print("Collision exists")
        time.sleep(5)
    else:   
        print("No collision")
    print("=====================================")

Debug:   RRTstar: Planner range detected to be 7.242362
Info:    RRTstar: Started planning with 1 states. Seeking a solution better than 0.00000.
Info:    RRTstar: Initial k-nearest value of 310
Info:    RRTstar: Found an initial solution with a cost of 1.78 in 20 iterations (21 vertices in the graph)
RRTstar found solution of path length 1.7772 with an optimization objective value of 1.7772Info:    RRTstar: Created 962 new states. Checked 463203 rewire options. 1 goal states in tree. Final solution cost 1.777

Interpolate Path length to 30
translation_diff =  2.7479052849065212e-08
rotation_diff =  0.0
checker: True
找到了 camera_link 的索引： 19
20
No collision
Debug:   RRTstar: Planner range detected to be 7.242362
Info:    RRTstar: Started planning with 1 states. Seeking a solution better than 0.00000.
Info:    RRTstar: Initial k-nearest value of 310
Info:    RRTstar: Found an initial solution with a cost of 1.54 in 3 iterations (4 vertices in the graph)
RRTstar found solution of path len

: 

: 

base_link
shoulder_1_link
arm_1_link
arm_2_link
wrist_1_link
wrist_2_link
wrist_3_link
flange_link
robotiq_arg2f_base_link
left_outer_knuckle
left_outer_finger
left_inner_finger
left_inner_finger_pad
left_inner_knuckle
right_outer_knuckle
right_outer_finger
right_inner_finger
right_inner_finger_pad
right_inner_knuckle
camera_link
找到了 camera_link 的索引： 19
20
-0.04308884716993392
camera link 和連結 arm_2_link 的最近距離小於 0.0 米
Collision exists
